# Tutorial 11: Adaptive RAG

Build a RAG system that **routes queries** to the optimal retrieval strategy based on question type.

**What you'll learn:**
- **Query Classification**: Categorize questions by type
- **Strategy Selection**: Route to appropriate retrieval method
- **Multiple Sources**: Vector store, web search, or direct LLM
- **Intelligent Routing**: Dynamic path selection

## Why Adaptive RAG?

Not all questions need the same approach:
- **Factual about documents**: Use vector store
- **Current events**: Use web search
- **Simple/general**: Direct LLM response

Adaptive RAG routes to the best strategy:

```
Question → Classify → Route → [VectorStore | WebSearch | Direct] → Generate
```

In [ ]:
# Setup
from langgraph_ollama_local import LocalAgentConfig
from langchain_ollama import ChatOllama

config = LocalAgentConfig()
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,
)
print(f"Using model: {config.ollama.model}")

In [ ]:
from typing import List, Literal
from typing_extensions import TypedDict
from langchain_core.documents import Document

class AdaptiveRAGState(TypedDict):
    """State for Adaptive RAG."""
    question: str
    query_type: Literal["vectorstore", "websearch", "direct"]  # Classified type
    documents: List[Document]
    generation: str

print("State defined!")

In [ ]:
from langgraph_ollama_local.rag.graders import QueryRouter

# Create query router
router = QueryRouter(llm)

# Test routing
test_queries = [
    "What is Self-RAG according to the research papers?",
    "What happened in the news today?",
    "What is 2 + 2?",
    "Explain the CRAG pattern from the documentation",
]

print("Query routing examples:")
for q in test_queries:
    route = router.route(q)
    print(f"  '{q[:40]}...' → {route}")

In [ ]:
from langgraph_ollama_local.rag import LocalRetriever

retriever = LocalRetriever()

# Node functions
def classify_query(state: AdaptiveRAGState) -> dict:
    """Classify the query type."""
    print("--- CLASSIFY QUERY ---")
    query_type = router.route(state["question"])
    print(f"Query type: {query_type}")
    return {"query_type": query_type}

def retrieve_vectorstore(state: AdaptiveRAGState) -> dict:
    """Retrieve from vector store."""
    print("--- VECTOR STORE RETRIEVAL ---")
    docs = retriever.retrieve_documents(state["question"], k=4)
    print(f"Retrieved {len(docs)} documents")
    return {"documents": docs}

def retrieve_websearch(state: AdaptiveRAGState) -> dict:
    """Retrieve from web search."""
    print("--- WEB SEARCH ---")
    # Simplified web search (use CRAG implementation for full version)
    return {"documents": [Document(
        page_content=f"Web search results for: {state['question']}",
        metadata={"type": "web"}
    )]}

def generate_direct(state: AdaptiveRAGState) -> dict:
    """Generate direct response without retrieval."""
    print("--- DIRECT GENERATION ---")
    response = llm.invoke(state["question"])
    return {"generation": response.content, "documents": []}

def generate_with_context(state: AdaptiveRAGState) -> dict:
    """Generate response using retrieved context."""
    print("--- GENERATE WITH CONTEXT ---")
    context = "\n\n".join([d.page_content for d in state["documents"]])
    prompt = f"Context:\n{context}\n\nQuestion: {state['question']}\n\nAnswer:"
    response = llm.invoke(prompt)
    return {"generation": response.content}

print("Nodes defined!")

In [ ]:
def route_query(state: AdaptiveRAGState) -> str:
    """Route based on query classification."""
    return state["query_type"]

print("Router defined!")

In [ ]:
from langgraph.graph import StateGraph, START, END

# Build graph
graph = StateGraph(AdaptiveRAGState)

# Add nodes
graph.add_node("classify", classify_query)
graph.add_node("vectorstore", retrieve_vectorstore)
graph.add_node("websearch", retrieve_websearch)
graph.add_node("direct", generate_direct)
graph.add_node("generate", generate_with_context)

# Add edges
graph.add_edge(START, "classify")

# Route based on classification
graph.add_conditional_edges(
    "classify",
    route_query,
    {
        "vectorstore": "vectorstore",
        "websearch": "websearch",
        "direct": "direct",
    }
)

# Retrieval nodes go to generate
graph.add_edge("vectorstore", "generate")
graph.add_edge("websearch", "generate")

# All paths end
graph.add_edge("direct", END)
graph.add_edge("generate", END)

adaptive_rag = graph.compile()
print("Adaptive RAG compiled!")

In [ ]:
# Visualize
from IPython.display import Image, display
try:
    display(Image(adaptive_rag.get_graph().draw_mermaid_png()))
except:
    print(adaptive_rag.get_graph().draw_ascii())

In [ ]:
# Test: Document question (vectorstore)
result1 = adaptive_rag.invoke({"question": "What is Self-RAG according to the papers?"})
print(f"Route: {result1['query_type']}")
print(f"Answer: {result1['generation'][:200]}...")

In [ ]:
# Test: Simple question (direct)
result2 = adaptive_rag.invoke({"question": "What is 2 + 2?"})
print(f"Route: {result2['query_type']}")
print(f"Answer: {result2['generation']}")

## Key Concepts

| Component | Purpose |
|-----------|--------|
| **QueryRouter** | Classifies question type |
| **Conditional Edges** | Routes to appropriate strategy |
| **Multiple Paths** | Different retrieval for different needs |

## What's Next?

In [Tutorial 12: Agentic RAG](12_agentic_rag.ipynb), you'll learn:
- Multi-step retrieval with an agent loop
- Query decomposition
- Iterative refinement